그라디오 챗봇와 랭체인 LLM 연동하기

In [1]:
# 그라디오 라이브러리를 설치합니다.
%pip install -q gradio 
# openai 라이브러리를 설치합니다.
%pip install -q openai 
# 랭체인 라이브러리를 설치합니다.
%pip install -q langchain 
# 랭체인 OpenAI 연동 라이브러리를 설치합니다.
%pip install -q -U langchain-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


#### LangChain의 ChatOpenAI가 환경 변수를 자동으로 로드하는 방식
* load_dotenv()를 실행하면 .env 파일에 있는 환경 변수가 시스템 환경 변수로 등록됨
* LangChain의 ChatOpenAI는 api_key=None일 때 os.getenv("OPENAI_API_KEY")를 자동으로 불러옴
* 따라서 api_key를 명시적으로 설정하지 않아도 API 키가 자동으로 적용됨

In [4]:
from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

True

In [2]:
import langchain

print(langchain.__version__)

0.3.18


In [7]:
from langchain_openai import ChatOpenAI

# LLM 인스턴스 생성
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

response = llm.invoke([{"role": "user", "content": "파이썬이란?"}])
print(response)

content='파이썬은 간결하고 읽기 쉽고 유연한 프로그래밍 언어로, 광범위한 응용 분야에서 사용되는 프로그래밍 언어입니다. Python으로 개발된 소프트웨어는 웹 개발, 데이터 분석, 인공지능, 머신러닝, 자동화 등 다양한 분야에서 사용됩니다. 파이썬은 초보자에게 친숙하며, 쉽게 배우고 사용할 수 있어서 많은 프로그래머들에게 인기가 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 169, 'prompt_tokens': 17, 'total_tokens': 186, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-7adcde07-a7ae-4cb5-b3e3-b1cdcb03bd3a-0' usage_metadata={'input_tokens': 17, 'output_tokens': 169, 'total_tokens': 186, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [10]:
print(response.content)

안녕하세요! 저는 인공지능 챗봇입니다. 무엇을 도와드릴까요?


#### API 키를 명시적으로 설정하는 방법

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일을 로드
load_dotenv()  

from langchain_openai import ChatOpenAI

# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")  
# 명시적으로 API 키 설정
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)  

response = llm.invoke([{"role": "user", "content": "파이썬이란?"}])
print(response)

content='파이썬은 인터프리터 방식으로 동작하는 고급 프로그래밍 언어이다. 발명자인 귀도 반 로섬(Guido van Rossum)에 의해 1991년에 발표되었으며, 파이썬의 주요 특징은 가독성과 간결성, 다양한 라이브러리와 모듈을 지원한다는 것이다. 파이썬은 데이터 분석, 인공지능, 웹 개발, 시스템 자동화 등 다양한 분야에서 널리 사용되고 있다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 170, 'prompt_tokens': 17, 'total_tokens': 187, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-a00a702a-b545-4eb1-921f-41121a44788b-0' usage_metadata={'input_tokens': 17, 'output_tokens': 170, 'total_tokens': 187, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
# 그라디오 라이브러리를 불러옵니다.
import gradio as gr
# LangChain 의 ChatOpenAI 를 불러옵니다.
from langchain_openai import ChatOpenAI

# ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(model="gpt-3.5-turbo")

# 채팅봇의 응답을 처리하는 함수를 정의합니다.
def chat_respond(message, chat_history):  
    response = llm.invoke([{"role": "user", "content": message}])
    bot_message = response.content
    
    chat_history.append({"role": "user", "content": message})
    chat_history.append({"role": "assistant", "content": bot_message})

    # 수정된 채팅 기록을 반환합니다.
    return "", chat_history  

# gr.Blocks()를 사용하여 인터페이스를 생성합니다.
with gr.Blocks() as demo:  
    chatbot = gr.Chatbot(label="채팅창", type="messages")  # '채팅창'이라는 레이블을 가진 채팅봇 컴포넌트를 생성합니다.
    msg = gr.Textbox(label="입력")  # '입력'이라는 레이블을 가진 텍스트박스를 생성합니다.
    clear = gr.Button("초기화")  # '초기화'라는 레이블을 가진 버튼을 생성합니다.

    # 텍스트박스에 메시지를 입력하고 제출하면 respond 함수가 호출되도록 합니다.
    msg.submit(chat_respond, [msg, chatbot], [msg, chatbot])  
    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화합니다.
    clear.click(lambda: None, None, chatbot, queue=False)  

# 인터페이스를 실행합니다. 실행하면 사용자는 '입력' 텍스트박스에 메시지를 작성하고 제출할 수 있으며, '초기화' 버튼을 통해 채팅 기록을 초기화 할 수 있습니다.
demo.launch(debug=True)  

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
